Collections
===

Problems that involve arrays, lists, and sequences... I borrow the term "Collections" from .NET. 

This is an incredibly broad category, so I may find that I wish to break this up into a new notebook in the future.

## 00 Find Median
### Description
...

### Solution
| In-place? | Type |
| -------- | -------- |
| Yes | `Span<int>` |

A solution to find the median of an array by hijacking a quicksort implementation. Note the use of `Span<T>` which provides type safety and memory safety without allocations when slicing the array.

I'd like to share some of this logic to write a proper quicksort, but since sorting is a "solved problem" for real-world purposes - I didn't bother. Note that searching for the median means that when splitting the array around your pivot, you only have to recurse down on the part you care about unlike a true quick sort. I also didn't bother making the method generic, which would be nice.

In [59]:
// extracts the median value from a Span<int>. mutates the Span<int>
static double QuickMedian(Span<int> span)
{
    // get partition pivot and ranges from Span<int>
    (int, Range, Range) GetPartition(Span<int> span)
    {
        int pivot = Partition(span);

        var front = (pivot + 1)..;
        var back = 0..pivot;

        return (pivot, front, back);
    }

    // partitions a Span<int> around the last element. mutates the Span<int>
    static int Partition(Span<int> span)
    {
        // set the pivot as the last element
        int pivot = span[^1];

        int forwardIndex = -1;

        // scan backwards and swap to orient values around pivot
        for (int backwardIndex = 0; backwardIndex < span.Length - 1; backwardIndex++)
        {
            if (span[backwardIndex] <= pivot)
            {
                (span[backwardIndex], span[++forwardIndex]) // swap
                    = (span[forwardIndex], span[backwardIndex]);
            }
        }

        // swap in pivot from end and return pivot
        (span[++forwardIndex], span[^1]) = (span[^1], span[forwardIndex]);

        return forwardIndex;
    }

    double QuickMedianEven(Span<int> span, int middle)
    {
        var (pivot, front, back) = GetPartition(span);

        return pivot switch
        {
            // if the pivot hits either median, just brute force to get the other
            var p when p == middle => (span[p] + span[front].ToArray().Min()) / 2d,
            var p when p == middle + 1 => (span[p] + span[back].ToArray().Max()) / 2d,
            // if pivot is < the median, choose the right partition (median is in right)
            var p when p < middle => QuickMedianEven(span[front], middle - (span.Length - span[front].Length)),
            // if pivot is > the median, choose the left partition
            _ => QuickMedianEven(span[back], middle)
        };
    }

    double QuickMedianOdd(Span<int> span, int middle)
    {
        var (pivot, front, back) = GetPartition(span);

        return pivot switch
        {
            var p when p == middle => span[middle], // we found the median
            // move left or right, as above
            var p when p < middle => QuickMedianOdd(span[front], middle - (span.Length - span[front].Length)),
            _ => QuickMedianOdd(span[back], middle)
        };
    }

    if (span is [])
        throw new EmptyArrayNotSupportedException();

    return span.Length % 2 == 0
        ?
        QuickMedianEven(span, (span.Length - 1) / 2)
        :
        QuickMedianOdd(span, (span.Length - 1) / 2);
}

public class EmptyArrayNotSupportedException : Exception
{
    public EmptyArrayNotSupportedException() : base("Empty arrays are not supported.")
    {
    }

    public EmptyArrayNotSupportedException(string message) : base(message)
    {
    }

    public EmptyArrayNotSupportedException(string message, Exception innerException) : base(message, innerException)
    {
    }
}

In [58]:
Enumerable.Range(1, 20)
    .Select(_ => Enumerable.Range(1, Random.Shared.Next(19) + 1)
        .Select(_ => Random.Shared.Next(20)).ToArray())
    .Select(arr => new { Array = arr, Median = QuickMedian(arr) })
    .DisplayTable()

Array,Median
"[ 0, 7, 5, 1, 9, 11, 13, 13, 17, 18, 18, 16, 15 ]",13
[ 8 ],8
"[ 1, 0, 3, 3, 6, 7, 7, 9, 9, 11, 12, 12, 14, 14, 17, 15, 17, 15 ]",10
"[ 0, 1, 2, 3, 7, 7, 7, 19, 11, 14 ]",7
"[ 3, 4, 8, 3, 10, 12, 13, 12, 11 ]",10
"[ 3, 1, 1, 2, 5, 6, 17, 14, 19, 9 ]",5.5
"[ 3, 3, 2, 3, 5, 6, 7, 8, 9, 14, 17, 19 ]",6.5
"[ 1, 1, 2, 5, 9, 15 ]",3.5
"[ 9, 9, 9, 11, 11 ]",9
[ 2 ],2
